Using the OR-Tools package by google

In [33]:
# Install OR-Tools if not installed
# %pip install ortools

from ortools.sat.python import cp_model

Making some sample data to show how it could work

In [34]:
# Sample data

exams = ['Math', 'Physics', 'Chemistry', 'History','Geography','PE','Music']
students = {
    'Alice': ['Math', 'Physics','Geography','PE'],
    'Bob': ['Physics', 'Chemistry','Music'],
    'Charlie': ['Math', 'History','PE'],
}
days = ['Mon', 'Tue', 'Wed','Thursday']

# Fixed exam slots (exam: day index)
fixed_slots = {
    'History': 2  # History exam fixed on Wed
}

Initialize the model

In [35]:

model = cp_model.CpModel()

In [36]:
# Create variables:
# exam_schedule[exam] = day assigned (0..len(days)-1)
exam_schedule = {}
for exam in exams:
    if exam in fixed_slots:
        # Fixed day
        exam_schedule[exam] = model.NewConstant(fixed_slots[exam])
    else:
        exam_schedule[exam] = model.NewIntVar(0, len(days)-1, exam)


The solver needs constraints, here are two simple ones to show what it can do

In [37]:

# Constraint 1: No student has more than 1 exam on the same day
for student, stu_exams in students.items():
    for i in range(len(stu_exams)):
        for j in range(i+1, len(stu_exams)):
            # exams for this student must be on different days
            model.Add(exam_schedule[stu_exams[i]] != exam_schedule[stu_exams[j]])

In [38]:

# Constraint 2: No more than 2 exams per day (optional example)
for d in range(len(days)):
    exams_on_day = []
    for exam in exams:
        is_on_day = model.NewBoolVar(f'{exam}_on_day_{d}')
        model.Add(exam_schedule[exam] == d).OnlyEnforceIf(is_on_day)
        model.Add(exam_schedule[exam] != d).OnlyEnforceIf(is_on_day.Not())
        exams_on_day.append(is_on_day)
    model.Add(sum(exams_on_day) <= 2)


The solver can then be ran to see if the problem can be solved

In [39]:
# Solve model
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.FEASIBLE or status == cp_model.OPTIMAL:
    print("Exam schedule:")
    for exam in exams:
        assigned_day = solver.Value(exam_schedule[exam])
        print(f" - {exam}: {days[assigned_day]}")
else:
    print("No solution found.")


Exam schedule:
 - Math: Mon
 - Physics: Wed
 - Chemistry: Tue
 - History: Wed
 - Geography: Tue
 - PE: Thursday
 - Music: Thursday
